# Lab 10: 2D DC Resistivity Data Collection
### Lillian Hanson, 5 November 2025

## Objectives:
- Collect two sets of 2D resistivity data over a buried concrete slab, and an architectural wall
- Use and familiarize ourselves with two pieces of equipment: the ABEM Terrameter and the AGI SuperSting
- Plot the pseudosections for each data collection array. (We will invert the data in the next lab period).

## Pseudosection Plots:

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import simpeg.electromagnetics.static.utils as static_utils

In [15]:
# standard sting column names
# source: https://www.agiusa.com/what-are-column-labels-supersting-stg-data-file
sting_columns = [
    'data record number',
    'USER',
    'date (YYYYMMDD)',
    'time (hh:mm:ss)',
    'V/I',
    '% error in tenths of percent',
    'output current in mA',
    'apparent resistivity in Ωm or Ωft',
    'command file identifier',
    'A(x)',
    'A(y)',
    'A(z)',
    'B(x)',
    'B(y)',
    'B(z)',
    'M(x)',
    'M(y)',
    'M(z)',
    'N(x)',
    'N(y)',
    'N(z)',
    'Cmd line number',
    'Transmitter volt code',
    '# of measurement cycles',
    'Measurement time used',
    'Gain setting',
    'Channel used',
]

### Archeology Wall Plots:

**Dipole-Dipole Plot:**

In [14]:
# load in data using standard STING column headers
arch_dipole_data = pd.read_csv('data/10-data/Wall_DPDP_1_perp.stg', skiprows=3, header=None, names=sting_columns)

A = np.c_[arch_dipole_data['A(x)'],arch_dipole_data['A(z)']]  # Will be an (n_data, 2) array
B = np.c_[arch_dipole_data['B(x)'],arch_dipole_data['B(z)']]
M = np.c_[arch_dipole_data['M(x)'],arch_dipole_data['M(z)']]
N = np.c_[arch_dipole_data['N(x)'],arch_dipole_data['N(z)']]

**Wenner Plot:**

### Dipping Slab Plots:

**Dipole-Dipole Plot:**

**Wenner Plot:**